In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')
import os
import collections

preDouble = "\\\\iobsdc01\\SharedDocs\\SEER_MEDICARE\\SEER_MEDICARE_STAGING"
new = os.chdir(preDouble)
print(preDouble)

\\iobsdc01\SharedDocs\SEER_MEDICARE\SEER_MEDICARE_STAGING


In [2]:
df = pd.read_pickle('dfpedsfFirsts.pickle')

## <font color='steelblue'>Description of the SEER-Medicare Data Files format</font>

Fixed width files. Data dictionary available to make the data not only machine readable but meaningful to humans

* Patient Entitlement and Diagnosis Summary File (PEDSF)
* Medicare Provider Analysis and Review (MEDPAR)
* Carrier Claims (old file name Physician/Supplier Part B (NCH))
* Outpatient Claims
* Home Health Agency (HAA)
* Hospice
* Durable Medical Equipment (DME)
* Medicare Part D Data

## <font color='steelblue'>First order of business is to identify all the patients in the SEER-Medicare data who have received PBT</font>

### <font color='steelblue'>Reminder:</font>

The relevant HCPCS are:

* 77520 - Proton trmt simple w/o comp
* 77522 - Proton trmt simple w/comp
* 77523 - Proton trmt intermediate
* 77525 - Proton treatment complex

as well as the HCPCS code S8030 (Scleral application of tantalum ring(s) for localization of lesions for proton beam therapy).

### <font color='steelblue'>Use the Outpatient Claims and Carrier Claims files</font>


* 218 Outpatient Claims files
* 207 Carrier Claims files


Loop through all of these files and capture the set of all the ```patient_id```s corresponding to the PBT HCPCS codes
